In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

# Load the dataset
df = pd.read_csv('dataset.csv')
precaution_df = pd.read_csv('symptom_precaution.csv')

# Preprocessing
df_with_precautions = df.merge(precaution_df, on='Disease')

df = pd.read_csv('dataset.csv')
df.head()
df1 = pd.read_csv('Symptom-severity.csv')
df1.head()
df2 = pd.read_csv('symptom_Description.csv')
df2.head()
df3 = pd.read_csv('symptom_precaution.csv')
df3.head()

df.isna().sum()
df.isnull().sum()
cols = df.columns
data = df[cols].values.flatten()
s = pd.Series(data)
s = s.str.strip()
s = s.values.reshape(df.shape)
df = pd.DataFrame(s, columns=df.columns)
df = df.fillna(0)
df.head()

vals = df.values
symptoms = df1['Symptom'].unique()
for i in range(len(symptoms)):
    vals[vals == symptoms[i]] = df1[df1['Symptom'] == symptoms[i]]['weight'].values[0]
d = pd.DataFrame(vals, columns=cols)
d = d.replace('dischromic _patches', 0)
d = d.replace('spotting_ urination',0)
df = d.replace('foul_smell_of urine',0)
df.head()

(df[cols] == 0).all()
df['Disease'].value_counts()
df['Disease'].unique()
data = df.iloc[:,1:].values
labels = df['Disease'].values

In [43]:
x_train_disease, x_test_disease, y_train_disease, y_test_disease = train_test_split(data, labels, test_size=0.2, random_state=42)

label_encoder = LabelEncoder()
y_train_disease = label_encoder.fit_transform(y_train_disease)
y_test_disease = label_encoder.transform(y_test_disease)

In [44]:
# Function to train and evaluate disease prediction model
def train_and_evaluate_disease_model(model, x_train, y_train, x_test, y_test):
    # Train the model
    model.fit(x_train, y_train)

    # Make predictions
    y_pred = model.predict(x_test)

    # Calculate F1-score
    f1 = f1_score(y_test, y_pred, average='macro')

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate confusion matrix
    confusion_mat = confusion_matrix(y_test, y_pred)

    return f1, accuracy, confusion_mat

# Initialize and train SVM model for disease prediction
svm_model_disease = SVC(kernel='linear')
f1_svm_disease, accuracy_svm_disease, confusion_mat_svm_disease = train_and_evaluate_disease_model(svm_model_disease, x_train_disease, y_train_disease, x_test_disease, y_test_disease)

In [46]:
x_precaution = df_with_precautions.drop(columns=['Disease', 'Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4'])
y_precaution = df_with_precautions[['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']]

# Splitting the data into train and test sets for precaution prediction
x_train_precaution, x_test_precaution, y_train_precaution, y_test_precaution = train_test_split(x_precaution, y_precaution, test_size=0.2, random_state=42)

y_train_precaution = label_encoder.fit_transform(y_train_precaution)
y_test_precaution = label_encoder.transform(y_test_precaution)

# Function to train and evaluate precaution prediction model
def train_and_evaluate_precaution_model(model, x_train, y_train, x_test, y_test):
    # Train the model
    model.fit(x_train, y_train)

    # Make predictions
    y_pred = model.predict(x_test)

    # Calculate F1-score
    f1 = f1_score(y_test, y_pred, average='macro')

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)

    return f1, accuracy

# Initialize and train SVM model for precaution prediction
svm_model_precaution = SVC(kernel='linear')
f1_svm_precaution, accuracy_svm_precaution = train_and_evaluate_precaution_model(svm_model_precaution, x_train_precaution, y_train_precaution, x_test_precaution, y_test_precaution)

# Function to predict disease and precautions
def predict_disease_and_precautions(symptoms):
    # Predict disease
    disease_prediction = svm_model_disease.predict([symptoms])

    # Predict precautions
    precaution_prediction = svm_model_precaution.predict([symptoms])

    return disease_prediction[0], precaution_prediction[0]

# Example usage
symptoms = [3, 5, 3, 5, 4, 4, 3, 2, 3, 0, 0, 0, 0, 0, 0, 0, 0]
disease, precautions = predict_disease_and_precautions(symptoms)
print("Predicted Disease:", disease)
print("Predicted Precautions:", precautions)

ValueError: y should be a 1d array, got an array of shape (3936, 4) instead.